In [1]:
%load_ext sql

In [2]:
%sql sqlite:///council.db

'Connected: @council.db'

In [3]:
%%sql 
PRAGMA table_info(CallsForProposals);

 * sqlite:///council.db
Done.


cid,name,type,notnull,dflt_value,pk
0,call_id,INTEGER,0,None,1
1,title,char(15),0,None,0
2,app_deadline,date,0,None,0
3,description,char(15),0,None,0
4,area,char(15),0,None,0
5,status,char(15),0,None,0


In [5]:
%%sql 
PRAGMA table_info(GrantProposals);

 * sqlite:///council.db
Done.


cid,name,type,notnull,dflt_value,pk
0,proposal_id,INTEGER,0,None,1
1,call_id,INTEGER,0,None,0
2,amount_requested,"DECIMAL(10, 2)",0,None,0
3,principal_investigator,INTEGER,0,None,0
4,application_status,CHAR(15),0,None,0
5,award_amount,"DECIMAL(10, 2)",0,None,0
6,award_date,DATE,0,None,0


In [42]:
%%sql
SELECT c.call_id, c.title
FROM CallsForProposals c
JOIN GrantProposals g ON c.call_id = g.call_id
WHERE g.amount_requested > 20000 OR (
        SELECT COUNT(*) + 1
        FROM Collaborators col
        WHERE col.proposal_id = g.proposal_id
      ) > 10
GROUP BY c.call_id, c.title
HAVING COUNT(DISTINCT g.proposal_id) > 0;

 * sqlite:///council.db
Done.


call_id,title
2,Chemistry Example
5,First Ever Proposal
9,Economics 103


In [41]:
%%sql
SELECT DISTINCT r_id FROM Reviewers 
    WHERE 
        conflicts_of_interest != (
            -- Primary Reviewer of the proposal
            SELECT principal_investigator FROM GrantProposals
            WHERE proposal_id = 5
        )
        AND conflicts_of_interest NOT IN (
            -- All reviewer ids with conflicting interests
            SELECT collaborator_id FROM Collaborators
            WHERE proposal_id = 5
        )
        AND r_id IN (
            -- All reviewers who have not reacher 3 review assignments
            SELECT reviewer_id FROM ReviewerAssignment
            GROUP BY reviewer_id
            HAVING COUNT(reviewer_id) < 3
        )
    ORDER BY r_id

 * sqlite:///council.db
Done.


r_id
1
3
4
5
8
9
10


In [ ]:
%%sql 
SELECT DISTINCT 

In [36]:
%%sql 
SELECT DISTINCT Re.firstname, Re.lastname FROM Researchers Re
JOIN Reviewers Rv ON Re.r_id = Rv.r_id

 * sqlite:///council.db
Done.


firstname,lastname
John,Doe
Michael,Johnson
Amanda,Lewis
Emily,Brown
David,Wilson
Jessica,Anderson
Matthew,Garcia


In [31]:
%%sql 
UPDATE GrantProposals SET award_date = '03-30-2024'
WHERE application_status = 'approved'


 * sqlite:///council.db
7 rows affected.


[]

In [22]:
%%sql
INSERT INTO GrantProposals (call_id, amount_requested, principal_investigator, application_status) 
VALUES (11, 25000, 10, 'pending');

 * sqlite:///council.db
1 rows affected.


[]

In [22]:
%%sql 
SELECT * FROM CallsForProposals;


 * sqlite:///council.db
Done.


call_id,title,app_deadline,description,area,status
1,Physics Example,2024-08-21,Description,Physics,completed
2,Chemistry Example,2024-09-21,Chemistry,Chemistry,completed
3,Another Proposal,2024-03-21,History!!,History,completed
4,Yet Another Proposal,2024-11-11,History!?,History,cancelled
5,First Ever Proposal,2024-10-24,History??,History,completed
6,Science Fiction Call,2024-05-24,Sociology Desc,Sociology,completed
7,Economics 101,1201-01-01,Postponed,Economics,postponed
8,Economics 102,2024-03-05,Postponed again,Economics,postponed
9,Economics 103,2019-01-01,We give up :(,Economics,cancelled
10,Never-Ending Call for Proposals,9999-12-31,The call that never ends!,Chemistry,ongoing


In [35]:
%%sql 
SELECT DISTINCT area FROM CallsForProposals;

 * sqlite:///council.db
Done.


area
Physics
Chemistry
History
Sociology
Economics


In [39]:
%%sql 
SELECT MAX (amount_requested) FROM GrantProposals G
JOIN CallsForProposals C ON G.call_id = C.call_id
WHERE C.area = 'Economics'

 * sqlite:///council.db
Done.


MAX (amount_requested)
1000000


In [66]:
%%sql 
SELECT * FROM GrantProposals;

 * sqlite:///council.db
Done.


proposal_id,call_id,amount_requested,principal_investigator,application_status,award_amount,award_date
1,10,10000,1,approved,8249,03-30-2024
2,10,4000,2,approved,2249,03-30-2024
3,9,100000,3,denied,None,None
4,1,10000,3,approved,8249,03-30-2024
5,10,9499,10,pending,None,None
6,9,15000,6,approved,13249,03-30-2024
7,6,100,6,approved,100,03-30-2024
8,9,6700,6,approved,4949,03-30-2024
9,10,84000,4,pending,None,None
10,9,9500,10,denied,None,None


In [65]:
%%sql
UPDATE GrantProposals SET award_amount = (award_amount - 1320) WHERE award_amount > 100

 * sqlite:///council.db
6 rows affected.


[]

In [45]:
%%sql 
SELECT G.award_amount, C.area FROM GrantProposals G
JOIN CallsForProposals C ON C.call_id = G.call_id

 * sqlite:///council.db
Done.


award_amount,area
10000,Chemistry
4000,Chemistry
None,Economics
10000,Physics
None,Chemistry
15000,Economics
100,Sociology
6700,Economics
None,Chemistry
None,Economics


In [21]:
%%sql 
UPDATE CallsForProposals SET app_deadline = '2024-0-24' 
WHERE call_id = 6;

 * sqlite:///council.db
1 rows affected.


[]

In [69]:
%%sql 
SELECT * FROM Reviewers

 * sqlite:///council.db
Done.


r_id,conflicts_of_interest
1,10
1,9
1,8
3,1
1,3
10,1
4,5
5,4
8,5
9,4


In [67]:
%%sql 
SELECT AVG(ABS(gp.amount_requested - gp.award_amount)) AS avg_discrepancy
FROM GrantProposals gp
JOIN CallsForProposals cp ON gp.call_id = cp.call_id
WHERE cp.area = 'Economics'


 * sqlite:///council.db
Done.


avg_discrepancy
1751.0
